In [ ]:
!pip install scanpy==1.9.1
!pip install matplotlib==3.6
!pip install -Iv rpy2==3.4.3
%load_ext rpy2.ipython

!pip install scib
!pip install anndata2ri


In [ ]:
%%R
if (!require("BiocManager", quietly = TRUE))
  install.packages("BiocManager")

BiocManager::install("S4Vectors")
BiocManager::install("SingleCellExperiment")

install.packages('Seurat')
library(Seurat)


In [ ]:
# convert datasets to Seurat objects for integration task
import os
import scipy
import scanpy as sc
import scib
import numpy as np
from pathlib import Path
import sys

# When using colab, set the path to the modules directory to use saved modules
sys.path.append('/content/drive/MyDrive/modules/')
from datasets_dict import datasets

# path to the original dataset (after subset to 3000 highly variable genes)
base_path = '/content/drive/MyDrive/Colab Notebooks/integrationDatasets/'
seurat_path = '/content/drive/MyDrive/Colab Notebooks/integrationDatasets/Seurat/originalObjects/'


# convert each dataset from anndata object to Seurat (.rds) object
# for dataset_name in datasets.keys():
for dataset_name in ['small_atac_windows']:
  
  # get dataset parameters
  batch_key = datasets[dataset_name]['batch_key']

  # set paths
  inPath = os.path.join(base_path, f"{dataset_name}_hvg.h5ad")
  outPath = os.path.join(seurat_path)

  # create directory if does not exists
  Path(outPath).mkdir(parents=True, exist_ok=True)

  # read the original dataset
  orig_data = sc.read(inPath)

  # make sure data is not sparse
  if scipy.sparse.issparse(orig_data.X):
    print("The given adata.X matrix is sparse. Converting to dense.")
    orig_data.X = np.asarray(orig_data.X.todense())

  # save Seurat (.rds) object
  print("Save as RDS")
  scib.preprocessing.saveSeurat(orig_data, f'{seurat_path}{dataset_name}_hvg.rds', batch_key)
  print("Converted: ", dataset_name)


